In [1]:
# 安装必要的依赖库
!pip install -q mlflow rouge_score transformers datasets evaluate
!pip install evaluate


# Import necessary toolkits
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback
from datasets import Dataset
import pandas as pd
import numpy as np
import torch
from evaluate import load as load_metric  # 使用 evaluate 库来加载指标
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import os

# 检查GPU是否可用
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [2]:
def preprocess_data(csv_file):
    df = pd.read_csv(csv_file)
    data = []

    # 根据实际列名设置输入列和标签列
    if 'text' in df.columns:
        text_column = 'text'
    elif 'sentence' in df.columns:
        text_column = 'sentence'
    elif 'prompt' in df.columns:
        text_column = 'prompt'
    elif 'baseq' in df.columns:
        text_column = 'baseq'
    elif 'question' in df.columns:
        text_column = 'question'
    else:
        raise ValueError(f"Column for text input not found in {csv_file}. Available columns are: {df.columns}")

    if 'label' not in df.columns:
        raise ValueError(f"Column 'label' not found in {csv_file}. Available columns are: {df.columns}")

    # 将每一行转换为适合T5模型的输入格式
    for _, row in df.iterrows():
        text = f"text: {row[text_column]}"  # 去掉 </s> 标记
        label = str(row['label'])  # 将标签转换为字符串形式
        data.append({"input_text": text, "target_text": label})

    return Dataset.from_pandas(pd.DataFrame(data))

In [3]:
BATCH_SIZE = 8
NUM_EPOCHS = 3
LEARNING_RATE = 5e-5

training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    logging_dir='./logs',
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    overwrite_output_dir=True,
    save_total_limit=3,
    load_best_model_at_end=True,  # 仅保留一个 load_best_model_at_end 参数
    fp16=True,
    generation_max_length=64,
    report_to="none",  # 设置早停
    metric_for_best_model="eval_loss",  # 评估指标为eval_loss
    greater_is_better=False,  # eval_loss越低越好
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [4]:
# 加载分类评估指标（例如 accuracy）
accuracy_metric = load_metric("accuracy")

# 定义评估函数
def compute_metrics(pred):
    pred_str = tokenizer.batch_decode(pred.predictions, skip_special_tokens=True)
    labels_ids = pred.label_ids
    labels_ids = np.where(labels_ids == -100, tokenizer.pad_token_id, labels_ids)
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    # 将字符串 'True'/'False' 转换为整数 1/0
    def str_to_int(label):
        if label.lower() == 'true':
            return 1
        elif label.lower() == 'false':
            return 0
        else:
            # 如果不是 'True' 或 'False'，尝试直接转换为整数
            try:
                return int(label)
            except ValueError:
                return -1  # 返回 -1 以便识别出无效值

    # 转换预测和标签为整数形式
    pred_int = [str_to_int(p) for p in pred_str]
    label_int = [str_to_int(l) for l in label_str]

    # 过滤掉无效的值
    valid_indices = [i for i, (p, l) in enumerate(zip(pred_int, label_int)) if p != -1 and l != -1]
    pred_int = [pred_int[i] for i in valid_indices]
    label_int = [label_int[i] for i in valid_indices]

    if not pred_int or not label_int:
        # 如果转换后没有有效的数据，返回空结果
        return {
            "accuracy": 0.0,
            "precision": 0.0,
            "recall": 0.0,
            "tp": 0,
            "tn": 0,
            "fp": 0,
            "fn": 0
        }

    # 计算混淆矩阵和分类指标
    tn, fp, fn, tp = confusion_matrix(label_int, pred_int).ravel()
    accuracy = accuracy_score(label_int, pred_int)
    precision = precision_score(label_int, pred_int, zero_division=0)
    recall = recall_score(label_int, pred_int, zero_division=0)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn
    }


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)

# 定义CSV文件路径列表
csv_files = [
    'df_toxicgen1_train_clean.csv',
    'ds_ethics_commonsense_small_test_clean.csv',
    'ds_ethics_commonsense_small_train_clean.csv',
    'ds_ethics_commonsense_small_validation_clean.csv',
    'ds_ethics_commonsense_test_clean.csv',
    'ds_ethics_commonsense_train_clean.csv',
    'ds_ethics_commonsense_validation_clean.csv',
    'HateSpeechDetection_Clean.csv',
    'HateSpeechDetection_simple_test_Clean.csv',
    'HateSpeechDetection_small_test_Clean.csv',
    'HateSpeechDetection_small_train_Clean.csv',
    'HateSpeechDetection_test_Clean.csv',
    'HateSpeechDetection_train_Clean.csv',
    'superset_simple_test.csv',
    'superset_test.csv',
    'superset_train.csv',
    'superset_train_small_test.csv',
    'superset_train_small_train.csv',
    'xstest_test_clean.csv',
    'xstest_train_clean.csv',
    'df_saladbench_attack_clean.csv',
    'df_saladbench_base_clean.csv',
    'df_saladbench_defense_clean.csv',
    'df_saladbench_ground_clean.csv',
    'df_tocxicchat1_simple_test_Clean.csv',
    'df_tocxicchat1_small_test_clean.csv',
    'df_tocxicchat1_small_train_clean.csv',
    'df_tocxicchat1_test_clean.csv',
    'df_tocxicchat1_train_clean.csv',
    'df_toxicgen1_simple_test_Clean.csv',
    'df_toxicgen1_small_test_clean.csv',
    'df_toxicgen1_small_train_clean.csv',
    'df_toxicgen1_test_clean.csv'
]

# 遍历每个CSV文件，分别进行训练和评估
results = []

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
for file_path in csv_files:
    print(f"Processing file: {file_path}")

    # 数据预处理
    dataset = preprocess_data(file_path)
    train_test_split = dataset.train_test_split(test_size=0.1)
    train_data = train_test_split['train']
    validation_data = train_test_split['test']

    # 加载模型到GPU
    model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

    # 数据集格式转换为模型输入
    def tokenize_data(batch):
        try:
            # 为输入和目标标记化
            tokenized_input = tokenizer(batch['input_text'], padding='max_length', truncation=True, max_length=512)
            tokenized_label = tokenizer(batch['target_text'], padding='max_length', truncation=True, max_length=64)

            # 设置标签
            tokenized_input['labels'] = tokenized_label['input_ids']
            return tokenized_input
        except Exception as e:
            print(f"Error in tokenization: {e}")
            return None

    # 对训练和验证数据进行标记化处理，并过滤掉 None 样本
    train_data = train_data.map(tokenize_data, batched=True, remove_columns=train_data.column_names)
    validation_data = validation_data.map(tokenize_data, batched=True, remove_columns=validation_data.column_names)

    # 过滤掉 None 样本
    train_data = train_data.filter(lambda x: x is not None)
    validation_data = validation_data.filter(lambda x: x is not None)

    # 初始化Trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_data,
        eval_dataset=validation_data,
    )

    # 训练模型
    try:
        trainer.train()
    except Exception as e:
        print(f"Error during training for file {file_path}: {e}")
        continue

    # 评估模型
    try:
        eval_results = trainer.evaluate()
    except Exception as e:
        print(f"Error during evaluation for file {file_path}: {e}")
        eval_results = {
            'eval_loss': None,
            'eval_accuracy': None,
            'eval_precision': None,
            'eval_recall': None,
            'eval_tp': None,
            'eval_tn': None,
            'eval_fp': None,
            'eval_fn': None
        }

    # 保存评估结果
    eval_results['file'] = file_path
    results.append(eval_results)

    # 打印结果
    print(f"Results for {file_path}:")
    print(eval_results)

# 记录所有文件的训练和评估结果
print("Final Results for All Files:")
for result in results:
    print(result)


Processing file: df_toxicgen1_train_clean.csv


Map:   0%|          | 0/8064 [00:00<?, ? examples/s]

Map:   0%|          | 0/896 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8064 [00:00<?, ? examples/s]

Filter:   0%|          | 0/896 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/94 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,Tp,Tn,Fp,Fn
1,0.290300,0.187035,0.000000,0.000000,0.000000,0,0,0,0
2,0.128700,0.039506,0.625000,0.000000,0.000000,0,10,0,6
3,0.056400,0.022622,0.638298,0.000000,0.000000,0,60,0,34


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Results for df_toxicgen1_test_clean.csv:
{'eval_loss': 0.022621579468250275, 'eval_accuracy': 0.6382978723404256, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_tp': 0, 'eval_tn': 60, 'eval_fp': 0, 'eval_fn': 34, 'eval_runtime': 2.5031, 'eval_samples_per_second': 37.553, 'eval_steps_per_second': 4.794, 'epoch': 3.0, 'file': 'df_toxicgen1_test_clean.csv'}
Final Results for All Files:
{'eval_loss': 0.007821546867489815, 'eval_accuracy': 0.7466517857142857, 'eval_precision': 0.6215139442231076, 'eval_recall': 0.5416666666666666, 'eval_tp': 156, 'eval_tn': 513, 'eval_fp': 95, 'eval_fn': 132, 'eval_runtime': 23.0916, 'eval_samples_per_second': 38.802, 'eval_steps_per_second': 4.85, 'epoch': 3.0, 'file': 'df_toxicgen1_train_clean.csv'}
{'eval_loss': 12.55142593383789, 'eval_accuracy': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_tp': 1, 'eval_tn': 1, 'eval_fp': 0, 'eval_fn': 0, 'eval_runtime': 0.1159, 'eval_samples_per_second': 17.26, 'eval_steps_per_second': 8.63, 'epoch': 3.0, '